In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Part II : Poisson Regression**

In [ ]:
# get the data
# transform days values to a range of integers
# transform the hours to one hot encoding 
# tansform the days to a label encoding
# introduce GLM theory
# fit a glm model to the data
# explain the results 
    # explain the p-values of the z-scores
    # explain the deviance test 
    # explain the goodness of fit test
# interpret the coefficients 
# plot the coefficents as a function of the hour and Base
# plot the counts and the predicted values Lambda against the days 
# plot a the Pearson residual against the fitted values
# identify the outliers and the points with high leverage and remove them
# identify what transformed version of the model can make the fit any better if the relationship is not linear
# do a bootstapping validation
    # for every version of the model :
        # fit the model onto 30 resampled subsets
        # get statistics like goodness of fit test values of all the 30 fits
        # draw a box plot of the values 

# make a conclusion about the best model

# fit a deep learning model
# calculate a goodness of fit of deep model

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
Uber = pd.read_csv('/kaggle/input/uber-pickups-summary/UberPickupsSummary (2).csv')
Uber = Uber.drop(columns=['Unnamed: 0'])
Uber

In [ ]:
encoded_labels, unique_values = pd.factorize(Uber['Date'])
Uber['Date'] = encoded_labels
Uber = pd.get_dummies(Uber, columns=['Base', 'hour']).drop(columns=['hour_0', 'Base_B02512'])
Uber['intercept'] = [1]*len(Uber)
Uber = Uber.astype(np.float32)
Uber

# An introduction to Generalize linear models Theorie

In simple linear regression , we regress the dependent variable Y onto the predictors X , using the formula $Y = X.\beta + \epsilon$ , such that Y is real number vector that follow as normal distribution $N(\mu,\sigma)$ and $\beta$ is the coefficients vector that we try to find by minimizing the the error vector $\epsilon$ using Least squares method, thus making the predicted values $X.\beta$ as close as possible to the real Y (i.e $Y \approx X.\beta$ ). 

Sounds great!, it turned out that this method doesn't work well when Y in not real number, this means that Y doesn't follow the normal distribution discribed above, so what can we do ? . To solve this problem we need go a step back to the linear regression problem and make small clarification that we will take as a starting point for our startegie.

When we said that $Y \approx X.\beta$ that was not precise enough,  in reality the correct formula should be $\mu = E[Y|X] \approx X.\beta$
this means that not every point Y is linear with the predictors X , but more precisely it is the conditional mean of Y for each value of X that should be in a linear relationship with X

With that being clarified,  we can start discovering the theoey of Generalized Linear Model "GLM"

**1. Modeling the probability distribution:**
When Y doesn't follow a normal distribution , we need first to figure what distribution it follows , one case is the classification problem where   $Y$ ~ $Bernoulli(p)$, where p is the mean of the distribution, using the idea discribed above about relationship of the mean and the predictors , instead of modeling $Y \approx X.\beta$, we can model 
$p \approx X.\beta$ , but $0 \leq p \leq 1$ ,which means we'll have some problems when modeling it with $X.\beta$  that ranges in $[-\infty, +\infty]$ , so we need to transforme p (or $X.\beta$) so that both sides of the equation can only have values from the same range, and then we model that tranformation as $$transform(p) \approx X.\beta$$.
The tranformation that is used in the classification problem is the one that is known as the **logit** tranformation
$$logit(p) = \log{\frac{p}{1-p}}$$


In [ ]:
y = Uber['size']
X = Uber.drop(columns=['size'])
model = sm.GLM(y, X, family = sm.families.Poisson())
results = model.fit()
print(results.summary())

# Explaining the test results

**1. The Deviance Test:**
The deviance test is analoguous to the F-test in linear regression
The very high values of the deviance $(1.1402*10^6)$ indicates that thers is enough eivdances to reject null hypothesis that $\beta = 0$ and accept the alternative that says that at least one of the coefficents is significantly different than zero


**2. The Z-test:**
The p-values of the z-scores are all lower than $5%$ Which shows that all the coefficents are significantly different than zero

**3. The Pearson $\chi^2$-test:**
This the goodness of fit test that tests how well the actual counts fit the predicted probabilities. $1.30*10^6$ is a large values for this test showing that there is a significant fit between the predicted probabilites and the actual Poisson count random variables in the columns 'size'


# Interpreting the Coefficients

We should be careful when interpreting the results, because an interpretation can differ when the coefficient of a categorical feature from when the feature has real values 

1. Coefficient of date feature:
the date feature in this dataset is an integer value tha can range from $0$ to $+\infty$ which means that our interpretation of the coefficient will the same as we do with a real values feature , in this case we can that $\beta_{Date} = 0.0036$ means that for every one day step to the future we expect an addition of approximitly 0.0036 in $E[Size]$ the expected value of  

In [ ]:
columns = ['hour_'+str(i) for i in range(1,24)]
hour_coefs = results.params[columns]
sns.relplot(x = range(1,24), y= hour_coefs.values, kind='line', marker='o')
plt.xlabel('hours')
plt.ylabel('count rate')
columns = ["Base_B02598", "Base_B02617", "Base_B02682", "Base_B02764"]
Base_coefs = results.params[columns]
sns.relplot(x = columns, y= Base_coefs.values, kind='line', marker='o')
plt.xlabel('base')
plt.ylabel('count rate')

In [ ]:
data = pd.DataFrame(results.fittedvalues, columns=['fitted_values'])
data['Date'] = X['Date']
data['counts'] = y.values
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data = data, x = 'Date', y='counts',ax=ax, label='counts')
data_grouped = data.groupby(by='Date').mean()
sns.lineplot(data = data_grouped, x='Date', y='fitted_values',ax=ax ,color='r', label='fitted_values')
plt.legend()

In [ ]:
resid = (data['counts'] - data['fitted_values'])
# /np.sqrt(data['fitted_values'])
fig, ax = plt.subplots()
sns.set_theme()
sns.scatterplot(x= data['fitted_values'], y= resid, ax=ax)
ax.set_xlabel('fitted_values')
ax.set_ylabel('residual')
ax.axhline(0, linewidth=1.3, color='red', linestyle='--')

In [ ]:
# algorithm
# models_list = [xgboost.XGRegressor, deepNN, SVR, GLM.poisson, KNN(1), KNN(k)]
# train each model on the train set
# generate 30 sample from the test set using bootstrapping 
# test each model in the model list on the 30 samples
# calculate the average chi2 square metric for each model's prediction
# drow a box plot of the calculated chi2 values 
# choose the best model

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

input_data = Uber.drop(columns=['size']).values
scaler = StandardScaler()
input_data = scaler.fit_transform(input_data)
output_data = Uber['size'].values

class PearsonGoodnessOfFit(tf.keras.metrics.Metric):
    def __init__(self, name='pearson chi squared distance', **kwargs):
        super().__init__(name=name, **kwargs)
        self.pearson_goodness_of_fit = self.add_weight(
            shape=(),
            initializer='zeros',
            name='pearson_goodness_of_fit'
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        

        if sample_weight is not None:
            pass

        self.pearson_goodness_of_fit.assign(tf.reduce_mean(tf.square(y_true - y_pred)/y_pred))

    def result(self):
        return self.pearson_goodness_of_fit

    
    
    
    
    
deep_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='exponential')
])

deep_model.compile(
    loss = tf.keras.losses.Poisson(),
    metrics = [tf.keras.metrics.MeanSquaredError()],
    optimizer = tf.keras.optimizers.Adam()
)

In [ ]:
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.stats import poisson
from scipy.stats import chisquare
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

train_set_x, test_set_x, train_set_y, test_set_y=train_test_split(input_data,output_data, test_size=0.2) 



def bootstrap_samples(test_set_x, test_set_y, n_samples):
    """
    Perform bootstrapping on two datasets.

    Parameters:
        test_set_x (array-like): The input data set X.
        test_set_y (array-like): The output data set Y.
        n_samples (int): Number of samples to generate.

    Returns:
        x_samples (list): List of bootstrapped samples from test_set_x.
        y_samples (list): List of bootstrapped samples from test_set_y.
    """
#     print(test_set_x)
    x_samples = []
    y_samples = []

    for _ in range(n_samples):
        # Generate random indices with replacement
        indices = np.random.randint(0, len(test_set_x), len(test_set_x))
#         print(indices)
        # Create bootstrapped samples for test_set_x and test_set_y
        x_bootstrap = test_set_x[indices] 
        y_bootstrap = test_set_y[indices] 

        # Append the bootstrapped samples to the lists
        x_samples.append(x_bootstrap)
        y_samples.append(y_bootstrap)

    return x_samples, y_samples






def chi_squared_distance(y_pred, y_true):
    """
    Calculate the Chi-squared distance between observed and expected values.

    Parameters:
        y_pred (array-like): Lambda parameter for Poisson distribution.
        y_true (array-like): Observed counts following Poisson distribution.

    Returns:
        chi2_dist (float): Chi-squared distance between observed and expected values.
    """
    y_pred = y_pred.reshape(-1,1)
#     print(np.sum(y_pred < 0))
    y_true = y_true.reshape(-1,1)
    
    # Calculate the expected probabilities using y_pred
    expected_probs = poisson.pmf(y_true, y_pred)
#     print(np.sum(np.isnan(expected_probs)))
    # Normalize the expected probabilities
#     expected_probs_normalized = expected_probs / np.sum(expected_probs)

    # Compute the Chi-squared distance
#     chi2_dist, _ = chisquare(y_true, f_exp=expected_probs_normalized)
    f_obs = np.ones(y_true.shape)
    chi2_dist = (f_obs - expected_probs)**2
    score = np.mean(chi2_dist)
#     print(score)
    return score




def draw_boxplot(scores_doubly_linked_list, model_names):
    """
    Draw a box plot of scores for different models on different test sets.

    Parameters:
        scores_doubly_linked_list (list): Doubly linked list containing lists of scores.
        model_names (list): Names of the models.

    Returns:
        None
    """
    
    # Flatten the doubly linked list to a list of scores and corresponding model names
    flat_scores = []
    flat_model_names = []
#     print(len(scores_doubly_linked_list))
#     print(len(model_names))
    for i in range(len(scores_doubly_linked_list)):
        for j in range(len(scores_doubly_linked_list[i])):
            flat_scores.append(scores_doubly_linked_list[i][j])
            flat_model_names.append(model_names[i]) 

    # Create a DataFrame from the flattened data
    df = pd.DataFrame({'Model': flat_model_names, 'Score': flat_scores})

    # Draw a box plot using Seaborn
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Model', y='Score', data=df)
    plt.title('Box Plot of Scores for Different Models')
    plt.xlabel('Model')
    plt.ylabel('Score')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
deep_model.fit(train_set_x, train_set_y, epochs=10, batch_size=5000, validation_split=0.2)

In [ ]:
# scm = []
# for i in range(len(test_sets_x)):
#     y_pred = deep_model.predict(test_sets_x[i])
#     y_true = np.array(test_sets_y[i]).reshape(-1,1)
# #     print(tf.keras.losses.MSE(test_sets_y[i], y_pred))
#     scm.append(np.mean(np.square(y_pred - y_true)))
# print(scm)
# scores.append(scm)

In [ ]:
# for i in range(len(test_sets_x)):
#     deep_model.evaluate(test_sets_x[i], test_sets_y[i])

In [ ]:
# deep_model.fit(train_set_x, train_set_y, epochs = 10, batch_size=5000, validation_split=0.2)

In [ ]:
svr = SVR()
xgb = XGBRegressor()
knn_1 = KNeighborsRegressor(n_neighbors=1)
knn_5 = KNeighborsRegressor(n_neighbors=5)

               
#                sm.GLM(train_y, train_X, family = sm.families.Poisson()) ]

# deep_model.fit(train_set_x, train_set_y, epochs = 10, batch_size=1000)
svr.fit(train_set_x, train_set_y)
xgb.fit(train_set_x, train_set_y)
knn_1.fit(train_set_x, train_set_y)
knn_5.fit(train_set_x, train_set_y)

In [ ]:
test_sets_x, test_sets_y = bootstrap_samples(test_set_x, test_set_y, 10)
print(test_sets_y[2].shape)

In [ ]:
models_list = [deep_model, 
#                svr,
#                xgb,
               knn_1,
               knn_5]
scores = []
for model in models_list:
    scm = []
    for i in range(len(test_sets_x)):
        y_true = np.array(test_sets_y[i]).reshape(-1,1)
        y_pred = model.predict(test_sets_x[i])
        y_pred = np.array(y_pred).reshape(-1,1)
#         print(np.mean(np.square(y_pred - test_sets_y[i])))
        scm.append(np.mean(np.square(y_pred - y_true)))
        
    scores.append(scm)
    



In [ ]:
print(scores)
draw_boxplot(scores ,['deep_model', 'KNN_1', 'KNN_5'])

In [ ]:
poisson.pmf(1, -1)